In [1]:
import ccxt
import pandas as pd
import time
from threading import Thread


In [2]:
# instantiation of exchange
exchange = ccxt.binanceusdm({
    'apiKey': '296015e94a672f83bfb4076908741a255cd5c9f192da047d12eb63cff4c73a7e',
    'secret': '4ecd9157f7b1a8486742374e9627d4ae4cfca3b997d1532ef447dc915fad62f5',
    'enableRateLimit': True,
})
exchange.set_sandbox_mode(True)


In [9]:

class CCXTTrader():
    def __init__(self, symbol, time_frame):
        self.symbol = symbol
        self.time_frame = time_frame
        self.get_available_intervals()

    def get_available_intervals(self):
        # get available intervals
        I = []
        for key, _ in exchange.timeframes.items():
            I.append(key)
        self.available_intervals = I

    # start_trading: get historical data and start streaming live data
    def start_trading(self, start=None, Lookback=None):
        if not Lookback:
            Lookback = 1000
        if self.time_frame in self.available_intervals:
            # 1. start collecting historical data with Lookback= 1000
            self.get_history(symbol=self.symbol,
                             interval=self.time_frame, limit=Lookback)
            # 2. start streaming live data
            thread = Thread(target=self.start_klines_stream, args=(
                self.stream_candles, self.symbol, self.time_frame))
            thread.start()

    # get most recent candles
    def get_history(self, symbol, interval, start=None, limit=1000):
        if start:
            start = exchange.parse8601(start)  # convert to milliseconds

        data = exchange.fetch_ohlcv(
            symbol=symbol, timeframe=interval, since=start, limit=limit)
        last_bar_actual = data[-1][0]  # timestamp of last loaded bar
        # timestamp of current bar
        last_bar_current = exchange.fetch_ohlcv(
            symbol=symbol, timeframe=interval, limit=1)[-1][0]
        # if lastBarActual != lastBarCurrent => pull the next 1000 bars
        while last_bar_actual != last_bar_current:
            time.sleep(0.1)
            data += exchange.fetch_ohlcv(symbol=symbol, timeframe=interval,
                                         since=last_bar_actual, limit=limit)[1:]
            last_bar_actual = data[-1][0]
            last_bar_current = exchange.fetch_ohlcv(
                symbol=symbol, timeframe=interval, limit=1)[-1][0]

        df = pd.DataFrame(data)
        df.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        df.Date = pd.to_datetime(df.Date, unit='ms')
        df.set_index('Date', inplace=True)
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        self.last_bar_time = df.index[-1]  # time@last_bar

        self.data = df

    # define streaming function
    def start_klines_stream(self, callback, symbol, interval):
        # global RUNNING  # global variable to control the loop externally
        self.running = True

        while self.running:
            res = exchange.fetch_ohlcv(
                symbol=symbol, timeframe=interval, limit=2)
            if len(res) == 0:
                print('no data')
            else:
                callback(res)
            time.sleep(1)

    # streaming controller
    def stop_stream(self):
        self.running = False

    # callback to handle streamed candles
    def stream_candles(self, res):
        # define how to process the streamed data

        # extract data from the response
        incoming_latest_time = pd.to_datetime(res[-1][0], unit='ms')
        first = res[-1][1]
        high = res[-1][2]
        low = res[-1][3]
        close = res[-1][4]
        volume = res[-1][5]
        
        # check if a bar is complete
        if incoming_latest_time == self.last_bar_time:
            complete = False
        else:  # a new bar is created => add the first bar # res[0]
            complete = True
            if len(res) == 2:
                self.data.loc[self.last_bar_time] = [res[0][1],
                                                     res[0][2], res[0][3], res[0][4], res[0][5], complete]
            else:
                self.data.loc[self.last_bar_time, 'Complete'] = complete
            self.last_bar_time = incoming_latest_time  # update the last_bar_time

        # print sth
        print('.', end='', flush=True)

        # feed self.data<df> with the latest completed bar
        self.data.loc[incoming_latest_time] = [first, high, low, close, volume, False]        

        # if a new bar is complete, do trading according to defined strategy
        if complete:
            print("\n", "Define Strategy and execute Trades")
            # TODO: define strategy and execute trades
            # self.define_strategy()
            # self.execute_trades()


In [10]:
trader = CCXTTrader(symbol='ETHUSDT', time_frame='1m')

In [11]:
trader.start_trading(start= None, Lookback= 10)

....................................................
 Define Strategy and execute Trades
........................................................
 Define Strategy and execute Trades
.........................................................
 Define Strategy and execute Trades
...

In [12]:
trader.stop_stream()

In [13]:
print(trader.data)

                        Open     High      Low    Close   Volume  Complete
Date                                                                      
2023-03-21 11:23:00  1728.99  1730.00  1707.22  1730.00   60.782      True
2023-03-21 11:24:00  1730.00  1730.00  1714.44  1714.44   36.747      True
2023-03-21 11:25:00  1714.44  1730.00  1705.46  1705.46   97.327      True
2023-03-21 11:26:00  1708.00  1716.51  1705.46  1705.46   14.160      True
2023-03-21 11:27:00  1705.46  1730.00  1705.46  1730.00   14.027      True
2023-03-21 11:28:00  1725.85  1730.00  1714.44  1719.94  148.132      True
2023-03-21 11:29:00  1725.85  1729.96  1720.92  1725.85    3.231      True
2023-03-21 11:30:00  1720.92  1730.00  1707.22  1730.00  264.343      True
2023-03-21 11:31:00  1721.75  1730.00  1705.46  1705.46  131.623      True
2023-03-21 11:32:00  1713.59  1714.44  1705.46  1713.59    7.896      True
2023-03-21 11:33:00  1705.46  1730.00  1705.46  1721.75  141.532      True
2023-03-21 11:34:00  1721